In [1]:
import pandas as pd
import ast
import os 
import multiprocessing as mp
import subprocess

In [9]:
dfc=pd.read_csv("corrected_cluster_info.tsv",sep="\t")


In [7]:
dfc.HGT_cluster_ID=dfc.index

In [10]:
dfc=pd.read_csv("corrected_cluster_info.tsv",sep="\t")

genes=[]
for index, row in dfc.iterrows():
    genes.extend(ast.literal_eval(row.all_cluster_chimeras))
    genes.append(row.representative_seq)

In [11]:
genes=set(genes)

In [14]:
!mkdir blast_round_one_data

In [25]:
genes=['XP_035715507.1', 'XP_037026007.1']

In [3]:
for g in genes:
    gx=[x for x in os.listdir("/net/bos-nfsisilon/ifs/rc_labs/extavour_lab/rkapoor/home_migrate/inter_blast_results") if g in x][0]
    !cp "/net/bos-nfsisilon/ifs/rc_labs/extavour_lab/rkapoor/home_migrate/inter_blast_results/""$gx" blast_round_one_data/"$g".tsv
        

In [4]:
for g in genes:
    df=pd.read_csv(f"blast_round_one_data/{g}.tsv",index_col=0,sep="\t",names=['qseqid', 'sseqid', 'stitle', 'staxids', 'sscinames', 'sphylums',
       'skingdoms', 'pident', 'length', 'mismatch', 'gapopen', 'qstart',
       'qend', 'sstart', 'send', 'evalue', 'bitscore'])
    df.to_csv(f"blast_round_one_data/{g}.tsv",sep="\t")

In [18]:
df=pd.read_csv("final_interval_sep_blasts_hgt/GCF_013339745.2;XP_049511280.1;HGT_(53,185).tsv",sep="\t",index_col=0)

In [6]:
metadf=pd.read_csv("meta_origin_final.csv",index_col=0)
metas=metadf.index
hgtdf=pd.read_csv("hgt_origin_final.csv",index_col=0)
hgtdf["gene"]=[x.split(";")[1] for x in hgtdf.index]
hgts=hgtdf.index

In [7]:
metas_2=[x for x in metas if x.split(";")[1] in genes]
hgts_2=[x for x in hgts if x.split(";")[1] in genes]
td=metas_2+hgts_2

In [14]:
!rm -r blast_round_two_data
!mkdir blast_round_two_data

In [27]:
td

['GCF_002217175.1;XP_035715507.1;Meta_(2,65)',
 'GCF_014529535.1;XP_037026007.1;Meta_(30,114)',
 'GCF_002217175.1;XP_035715507.1;HGT_(127,249)',
 'GCF_014529535.1;XP_037026007.1;HGT_(202,345)']

In [9]:
f=[]
for g in td:
    
    if "HGT_" in g:
        print(g)
        !cp "final_interval_sep_blasts_hgt/""$g".tsv blast_round_two_data/"$g".tsv
    else:
        !cp "final_interval_sep_blasts_meta/""$g".tsv blast_round_two_data/"$g".tsv

GCF_002217175.1;XP_035715507.1;HGT_(127,249)
GCF_014529535.1;XP_037026007.1;HGT_(202,345)


In [18]:
import pickle
file_path = 'chimera_intervals_final.pickle'
with open(file_path, 'rb') as file:
    chimera_intervals=pickle.load(file)

In [23]:
with open(file_path, 'wb') as file:
    pickle.dump(chimera_intervals, file)

In [20]:
chimera_intervals["XP_037026007.1"]=[x for x in td if "XP_037026007.1" in x]

In [22]:
chimera_intervals["XP_037026007.1"]

['GCF_014529535.1;XP_037026007.1;Meta_(30,114)',
 'GCF_014529535.1;XP_037026007.1;HGT_(202,345)']

In [17]:
!python3 blastplot2.py "XP_037026007.1"

hi
Traceback (most recent call last):
  File "/n/holylabs/LABS/extavour_lab/Users/rkapoor/pipeline_final/blastplot2.py", line 205, in <module>
    make_plotf(n)
  File "/n/holylabs/LABS/extavour_lab/Users/rkapoor/pipeline_final/blastplot2.py", line 121, in make_plotf
    ints=results2[n]
KeyError: 'XP_037026007.1'


In [16]:
!python3 blastplot2.py "XP_035715507.1"

hi


In [24]:
!python3 blastplot2.py "XP_037026007.1"
!python3 blastplot1.py "XP_037026007.1"

hi
hi


In [65]:
f=open("final_intervals.txt","w")
for g in genes:
    l=f"{g}:{str([x for x in td if g in x])}\n"
    f.write(l)
f.close()

In [78]:
chimera_map={}
for g in genes:
    chimera_map[g]=[x for x in td if g in x]


In [81]:
import pickle
with open("chimera_intervals_final.pickle", 'wb') as file:
    pickle.dump(chimera_map, file)


In [109]:
!rm -r hmmer_phylogenetics_results
!mkdir hmmer_phylogenetics_results

In [112]:
!ls "'hmmer_results_phylogenetics/GCF_000572035.2;XP_008551656.1;Meta_(624,958)"

ls: cannot access ''\''hmmer_results_phylogenetics/GCF_000572035.2;XP_008551656.1;Meta_(624,958)': No such file or directory


In [132]:
missing=set(td)-set(os.listdir("hmmer_results_phylogenetics"))
direct="/n/holylabs/LABS/extavour_lab/Users/rkapoor/hmmer_phylo_data"
missing2=[]
for n in missing:
    rdict=SeqIO.to_dict(SeqIO.parse(f"{direct}/{n}/sub_phylo_tax.fasta", "fasta"))
    df=pd.read_csv(f"{direct}/{n}/phylo_tax2.tsv",sep="\t",index_col=0)
    df=df[df.target_name.isin(["_".join(x.split("_")[0:-2]) for x in rdict.keys()])]
    if df[~(df["E-value"]>.01)].shape[0]!=len(rdict):
#         !cp -r "$direct"/"$n" "hmmer_phylogenetics_results/""$n"
        missing2.append(n)
    !cp -r "$direct"/"$n" "hmmer_results_phylogenetics/""$n"

In [4]:
for n in missing2:
    subprocess.run(["sbatch","align_iq_pipe_serial_requeue.sh",n])
    

Submitted batch job 17924704
Submitted batch job 17924705
Submitted batch job 17924706
Submitted batch job 17924707


In [1]:
missing2=['GCF_921293095.1;XP_046396626.1;HGT_(32,113)',
 'GCF_000648655.2;XP_023247543.1;HGT_(1269,1347)',
 'GCF_000648655.2;XP_023247543.1;Meta_(775,886)',
 'GCF_000591075.1;XP_023327453.1;HGT_(27,114)']

In [133]:
for n in missing2:
    rdict=SeqIO.to_dict(SeqIO.parse(f"hmmer_results_phylogenetics/{n}/sub_phylo_tax.fasta", "fasta"))
    df=pd.read_csv(f"hmmer_results_phylogenetics/{n}/phylo_tax2.tsv",sep="\t",index_col=0)
    df=df[df.target_name.isin(["_".join(x.split("_")[0:-2]) for x in rdict.keys()])]
    if df[~(df["E-value"]>.01)].shape[0]!=len(rdict):
        td=[x for x in rdict if "_".join(x.split("_")[0:-2]) in list(df[~(df["i-Evalue"]>.01)].target_name)]
        odict={x:rdict[x] for x in td}
        !mkdir -p "hmmer_phylogenetics_corrected/""$n"
        with open(f'hmmer_phylogenetics_corrected/{n}/sub_phylo_tax.fasta', 'w') as handle:
            SeqIO.write(odict.values(), handle, 'fasta')
        df[~(df["E-value"]>.01)].to_csv(f"hmmer_phylogenetics_corrected/{n}/phylo_tax2.tsv")
        
        td=[x for x in rdict if "_".join(x.split("_")[0:-2]) in list(df[~(df["i-Evalue"]>.01)].target_name)]
        f=open(f"hmmer_results_phylogenetics/{n}/labels.txt","r")
        f2=open(f"hmmer_phylogenetics_corrected/{n}/labels.txt","w")
        for x in f.readlines():
            if 'normal' in x:
                if x.split(",")[0] in td:
                    f2.write(x)
            else:
                f2.write(x)
            
        f2.close()
        f.close()
        f=open(f"hmmer_results_phylogenetics/{n}/labels2.txt","r")
        f2=open(f"hmmer_phylogenetics_corrected/{n}/labels2.txt","w")
        for x in f.readlines():
            if 'normal' in x:
                if x.split(",")[0] in td:
                    f2.write(x)
            else:
                f2.write(x)
            
        f2.close()
        f.close()
        f=open(f"hmmer_results_phylogenetics/{n}/colorstrip.txt","r")
        f2=open(f"hmmer_phylogenetics_corrected/{n}/colorstrip.txt","w")
        for x in f.readlines():
            if " #" in x and "MANDATORY" not in x:
                if x.split(" ")[0] in td:
                    print(x)
                    f2.write(x)
            else:
                f2.write(x)

        f2.close()
        f.close()


OGR97248.1_13_107 #0000ff

OIO00285.1_14_108 #0000ff

OGR66501.1_12_108 #0000ff

OGR44198.1_12_107 #0000ff

OGR48438.1_11_104 #0000ff

NLI09560.1_12_113 #0000ff

PKN01716.1_14_111 #0000ff

MBP6940175.1_11_108 #0000ff

MCI6590461.1_14_105 #0000ff

HIV09905.1_13_106 #0000ff

MBR4317478.1_19_108 #0000ff

MCF8045496.1_13_107 #0000ff

NCU33045.1_2_95 #0000ff

GIV14111.1_323_404 #0000ff

WP_008908814.1_59_141 #0000ff

WP_274767293.1_14_96 #0000ff

WP_110940361.1_57_142 #0000ff

CAD7241107.1_20_101 #ff0000

MBP7796746.1_11_111 #0000ff

OGV59991.1_12_109 #0000ff

MBN1411124.1_28_116 #0000ff

OGV43487.1_64_148 #0000ff

OGV54756.1_63_151 #0000ff

MBR5532863.1_1_86 #0000ff

MCF8433513.1_6_108 #0000ff

UDQ97639.1_15_111 #0000ff

KPK90608.1_13_110 #0000ff

MBI3457589.1_16_96 #0000ff

OQY97504.1_5_94 #0000ff

XP_050573467.1_35_112 #ff0000

XP_012163346.2_33_111 #ff0000

XP_033188210.1_32_111 #ff0000

XP_011686572.1_23_120 #ffc0cb

XP_018299621.1_23_102 #ff0000

XP_011304692.1_33_113 #ffc0cb

XP_0151

In [16]:
!rm -r "hmmer_phylogenetics_results"
!mkdir "hmmer_phylogenetics_results"

In [42]:
for x in td:
    if x in os.listdir("hmmer_phylogenetics_corrected"):
        print(x)
        !mkdir -p "hmmer_phylogenetics_results/""$x"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"phylo_tax2.tsv" "hmmer_phylogenetics_results"/"$x"/"filtered_taxonomic_annotated_hmmsearch_result.tsv"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"MSA_hmm_output_final.fasta" "hmmer_phylogenetics_results"/"$x"/"MSA.fasta"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"trimmed_MSA_hmm_output_final.fasta" "hmmer_phylogenetics_results"/"$x"/"trimmed_MSA.fasta"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile" "hmmer_phylogenetics_results"/"$x"/"iqtree_result.treefile" 
        try:
            !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile.rooted2" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        except:
            !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile.rooted" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"labels.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_taxonomic.txt"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"labels.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_taxonomic.txt"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"labels2.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_sequence_description.txt"
        !cp "hmmer_phylogenetics_corrected"/"$x"/"colorstrip.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_colorstrip.txt"

    else:

        !mkdir -p "hmmer_phylogenetics_results/""$x"
        !cp "hmmer_results_phylogenetics"/"$x"/"phylo_tax2.tsv" "hmmer_phylogenetics_results"/"$x"/"filtered_taxonomic_annotated_hmmsearch_result.tsv"
        !cp "hmmer_results_phylogenetics"/"$x"/"MSA_hmm_output_final.fasta" "hmmer_phylogenetics_results"/"$x"/"MSA.fasta"
        !cp "hmmer_results_phylogenetics"/"$x"/"trimmed_MSA_hmm_output_final.fasta" "hmmer_phylogenetics_results"/"$x"/"trimmed_MSA.fasta"
        !cp "hmmer_results_phylogenetics"/"$x"/"rev_aa.treefile" "hmmer_phylogenetics_results"/"$x"/"iqtree_result.treefile" 
        try:
            !cp "hmmer_results_phylogenetics"/"$x"/"rev_aa.treefile.rooted2" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        except:
            !cp "hmmer_results_phylogenetics"/"$x"/"rev_aa.treefile.rooted" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        !cp "hmmer_results_phylogenetics"/"$x"/"labels.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_taxonomic.txt"
        !cp "hmmer_results_phylogenetics"/"$x"/"labels.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_taxonomic.txt"
        !cp "hmmer_results_phylogenetics"/"$x"/"labels2.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_sequence_description.txt"
        !cp "hmmer_results_phylogenetics"/"$x"/"colorstrip.txt" "hmmer_phylogenetics_results"/"$x"/"itol_annotation_colorstrip.txt"
    !cp "hmmer_results_phylogenetics"/"$x"/"hmmer_result.tsv" "hmmer_phylogenetics_results"/"$x"/"hmmer_result.tsv"
    !cp "hmmer_results_phylogenetics"/"$x"/"secondary_chimera.tsv" "hmmer_phylogenetics_results"/"$x"/"secondary_chimera.tsv"


GCF_002217175.1;XP_035715507.1;HGT_(127,249)
GCF_014529535.1;XP_037026007.1;HGT_(202,345)


In [46]:
df=pd.read_csv("hmmer_phylogenetics_results/GCF_014529535.1;XP_037026007.1;HGT_(202,345)/filtered_taxonomic_annotated_hmmsearch_result.tsv",index_col=0,sep="\t")

In [47]:
df

""
",taxid,Unnamed: 0,species,target_name,target_accession,tlen,query name,accession,qlen,E-value,score1,bias1,#,of,c-Evalue,i-Evalue,score,bias,hmmfrom,hmmto,alifrom,alito,envfrom,envto,acc,description of target,chimera,superkingdom,kingdom,phylum,order,sci_name,color"
"0,38358.0,0,Bradysia coprophila,XP_037026007.1,,,,,,,,,,,,,,,,,,,192.0,355.0,,XP_037026007.1 uncharacterized protein LOC119067246 [Bradysia coprophila],True,Eukaryota,Metazoa,Arthropoda,Diptera,Bradysia coprophila,pink"
"323,1564500.0,147,Bradysia odoriphaga,KAG4070553.1,-,323.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,9.300000000000001e-86,299.9,2.2,1.0,1.0,9.899999999999999e-91,9.300000000000001e-86,299.9,2.2,1.0,150.0,143.0,292.0,143.0,292.0,1.0,hypothetical protein HA402_001219 [Bradysia odoriphaga],False,Eukaryota,Metazoa,Arthropoda,Diptera,Bradysia odoriphaga,pink"
"264,1174678.0,671,Morchella snyderi,KAI5844493.1,-,248.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,1.1e-15,72.6,0.1,1.0,1.0,1.5e-20,1.4e-15,72.1,0.1,2.0,150.0,66.0,217.0,65.0,217.0,0.87,hypothetical protein DFP73DRAFT_549515 [Morchella snyderi],False,Eukaryota,Fungi,Ascomycota,Pezizales,Morchella snyderi,brown"
"66,60458.0,1100,Tuber brumale,KAG0642390.1,-,270.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,5.4e-15,70.3,0.1,1.0,1.0,7.5e-20,7e-15,69.9,0.1,2.0,147.0,88.0,234.0,87.0,237.0,0.88,hypothetical protein HOY80DRAFT_639808 [Tuber brumale],False,Eukaryota,Fungi,Ascomycota,Pezizales,Tuber brumale,brown"
"123,160517.0,1068,Tirmania nivea,KAF8418308.1,-,316.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,7.1e-15,69.9,0.0,1.0,1.0,1e-19,9.6e-15,69.4,0.0,2.0,150.0,130.0,280.0,129.0,280.0,0.86,hypothetical protein EV426DRAFT_323777 [Tirmania nivea],False,Eukaryota,Fungi,Ascomycota,Pezizales,Tirmania nivea,brown"
...
"86,102119.0,1042,Synechococcus sp. C9,WP_247216244.1,-,204.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,0.008,30.7,0.1,1.0,1.0,8e-07,0.074,27.6,0.0,60.0,135.0,84.0,167.0,59.0,174.0,0.8,hypothetical protein [Synechococcus sp. C9],False,Bacteria,None,Cyanobacteriota,Synechococcales,Synechococcus sp. C9,blue"
"841,2771122.0,627,Microcystis sp. M055S1,MCA2873135.1,-,202.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,0.0022,32.6,0.0,1.0,1.0,2e-06,0.19,26.3,0.0,51.0,133.0,82.0,170.0,51.0,184.0,0.79,hypothetical protein [Microcystis sp. M055S1],False,Bacteria,None,Cyanobacteriota,Chroococcales,Microcystis sp. M055S1,blue"
"205,656916.0,95,Aspergillus arachidicola,KAE8343887.1,-,670.0,""GCF_014529535.1;XP_037026007.1;HGT_(202,345)"",-,150.0,0.003,32.1,0.0,1.0,1.0,2.9e-06,0.27,25.8,0.0,64.0,135.0,546.0,624.0,521.0,632.0,0.82,hypothetical protein BDV24DRAFT_161190 [Aspergillus arachidicola],False,Eukaryota,Fungi,Ascomycota,Eurotiales,Aspergillus arachidicola,brown"


In [49]:
for x in td:
    try:
        df=pd.read_csv(f"hmmer_phylogenetics_results/{x}/filtered_taxonomic_annotated_hmmsearch_result.tsv",index_col=0)
        df.to_csv(f"hmmer_phylogenetics_results/{x}/filtered_taxonomic_annotated_hmmsearch_result.tsv",sep="\t")
    except:
        print(x)

GCF_002217175.1;XP_035715507.1;Meta_(2,65)
GCF_014529535.1;XP_037026007.1;Meta_(30,114)


In [50]:
for x in os.listdir("hmmer_phylogenetics_results"):
    tr=[y for y in os.listdir(f"hmmer_phylogenetics_results/{x}") if ".csv" in y]
    for ti in tr:
        !rm hmmer_phylogenetics_results/"$x"/"$ti"
                        

In [55]:
td

['GCF_002217175.1;XP_035715507.1;Meta_(2,65)',
 'GCF_014529535.1;XP_037026007.1;Meta_(30,114)',
 'GCF_002217175.1;XP_035715507.1;HGT_(127,249)',
 'GCF_014529535.1;XP_037026007.1;HGT_(202,345)']

In [53]:
df=pd.read_csv("hmmer_phylogenetics_results/GCF_905147365.1;XP_041972388.1;HGT_(56,486)/hmmer_result.tsv",sep="\t",index_col=0)
cols=list(df.columns)

In [54]:
for x in td:
    df=pd.read_csv(f"hmmer_phylogenetics_results/{x}/hmmer_result.tsv",index_col=0,sep="\t",names=cols)
    df.to_csv(f"hmmer_phylogenetics_results/{x}/hmmer_result.tsv",sep="\t")
 

In [56]:
df

,target_accession,tlen,query_name,accession,qlen,E-value,score1,bias1,#,of,...,bias,hmmfrom,hmmto,alifrom,alito,envfrom,envto,acc,description_of_target,species
XP_037045975.1,-,279,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,9.300000e-86,299.9,1.8,1,1,...,1.8,1,150,99,248,99,248,1.00,uncharacterized protein LOC119081310 [Bradysia...,Bradysia coprophila
KAG4070553.1,-,323,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,9.300000e-86,299.9,2.2,1,1,...,2.2,1,150,143,292,143,292,1.00,hypothetical protein HA402_001219 [Bradysia od...,Bradysia odoriphaga
XP_037026007.1,-,395,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,1.500000e-59,214.9,0.3,1,1,...,0.3,1,150,202,349,202,349,0.99,uncharacterized protein LOC119067246 [Bradysia...,Bradysia coprophila
XP_045970291.1,-,398,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,3.600000e-27,109.8,0.0,1,2,...,0.0,47,150,12,116,1,116,0.83,uncharacterized protein LAJ45_06962 [Morchella...,Morchella importuna
XP_045970291.1,-,398,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,3.600000e-27,109.8,0.0,2,2,...,0.0,2,148,211,360,210,362,0.74,uncharacterized protein LAJ45_06962 [Morchella...,Morchella importuna
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_008317519.1,-,213,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,9.900000e+00,20.7,0.0,1,1,...,0.0,57,135,98,174,71,180,0.81,type I restriction enzyme HsdR N-terminal doma...,Leptolyngbya sp. PCC 6406
HAO20149.1,-,200,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,1.000000e+01,20.7,0.0,1,1,...,0.0,32,134,61,167,47,179,0.68,hypothetical protein [Desulfobacteraceae bacte...,Desulfobacteraceae bacterium
HAG83903.1,-,205,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,1.000000e+01,20.7,0.0,1,1,...,0.0,55,138,98,175,55,184,0.75,hypothetical protein [Cyanobacteria bacterium ...,Cyanobacteria bacterium UBA12227
CAG8814548.1,-,123,"GCF_014529535.1;XP_037026007.1;HGT_(202,345)",-,150,1.000000e+01,20.7,0.9,1,2,...,0.1,8,44,1,37,1,40,0.88,"13083_t:CDS:2, partial [Gigaspora rosea]",Gigaspora rosea


In [1]:
import os

In [29]:
for x in td:
    !rm "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
    if x in os.listdir("hmmer_phylogenetics_corrected"):
        !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile" "hmmer_phylogenetics_results"/"$x"/"iqtree_result.treefile"
        try:
            !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile.rooted2" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        except:
            print(x)


    else:
        !cp "hmmer_results_phylogenetics"/"$x"/"rev_aa.treefile" "hmmer_phylogenetics_results"/"$x"/"iqtree_result.treefile"
        try:
            !cp "hmmer_results_phylogenetics"/"$x"/"rev_aa.treefile.rooted2" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
        except:
            print(x) 

In [3]:
for x in missing2:
    try:
        !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile.rooted2" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"
    except:
        !cp "hmmer_phylogenetics_corrected"/"$x"/"rev_aa.treefile.rooted" "hmmer_phylogenetics_results"/"$x"/"MAD_rooted_iqtree_result.treefile"

In [1]:
!zip -r blast_round_one_data.zip blast_round_one_data

  adding: blast_round_one_data/ (stored 0%)
  adding: blast_round_one_data/XP_023247543.1.tsv (deflated 82%)
  adding: blast_round_one_data/XP_044009448.1.tsv (deflated 82%)
  adding: blast_round_one_data/XP_037790819.1.tsv (deflated 80%)
  adding: blast_round_one_data/XP_045595407.1.tsv (deflated 76%)
  adding: blast_round_one_data/XP_031781467.1.tsv (deflated 83%)
  adding: blast_round_one_data/XP_041972388.1.tsv (deflated 78%)
  adding: blast_round_one_data/XP_049511280.1.tsv (deflated 84%)
  adding: blast_round_one_data/XP_034487048.1.tsv (deflated 79%)
  adding: blast_round_one_data/XP_037049533.1.tsv (deflated 82%)
  adding: blast_round_one_data/XP_049881676.1.tsv (deflated 81%)
  adding: blast_round_one_data/XP_037943392.1.tsv (deflated 83%)
  adding: blast_round_one_data/XP_027204138.1.tsv (deflated 84%)
  adding: blast_round_one_data/XP_015837071.1.tsv (deflated 82%)
  adding: blast_round_one_data/XP_034245505.1.tsv (deflated 83%)
  adding: blast_round_one_data/.ipynb_checkpoi

In [2]:
!zip -r blast_round_two_data.zip blast_round_two_data

  adding: blast_round_two_data/ (stored 0%)
  adding: blast_round_two_data/GCF_010883055.1;XP_033209845.1;Meta_(1663,2170).tsv (deflated 80%)
  adding: blast_round_two_data/GCF_014529535.1;XP_037030969.1;HGT_(226,299).tsv (deflated 88%)
  adding: blast_round_two_data/GCF_000599845.2;XP_023318028.1;HGT_(1095,1213).tsv (deflated 83%)
  adding: blast_round_two_data/GCF_004193835.1;XP_028168683.1;Meta_(344,414).tsv (deflated 86%)
  adding: blast_round_two_data/GCF_000699045.2;XP_018326454.1;HGT_(14,94).tsv (deflated 82%)
  adding: blast_round_two_data/GCF_015228065.1;XP_037790819.1;Meta_(45,640).tsv (deflated 82%)
  adding: blast_round_two_data/GCF_010883055.1;XP_033212818.1;Meta_(722,1217).tsv (deflated 79%)
  adding: blast_round_two_data/GCF_921293095.1;XP_046403459.1;Meta_(26,123).tsv (deflated 85%)
  adding: blast_round_two_data/GCF_021134715.1;XP_046456333.1;Meta_(1402,1560).tsv (deflated 85%)
  adding: blast_round_two_data/GCF_025091365.1;XP_050528041.1;Meta_(271,749).tsv (deflated 8

In [3]:
!zip -r hmmer_phylogenetics_results.zip hmmer_phylogenetics_results

updating: hmmer_phylogenetics_results/ (stored 0%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/ (stored 0%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/MAD_rooted_iqtree_result.treefile (deflated 56%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/itol_annotation_sequence_description.txt (deflated 72%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/itol_annotation_taxonomic.txt (deflated 76%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/filtered_taxonomic_annotated_hmmsearch_result.tsv (deflated 78%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/iqtree_result.treefile (deflated 55%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645476.1;HGT_(93,258)/itol_annotation_colorstrip.txt (deflated 66%)
updating: hmmer_phylogenetics_results/GCF_021234035.1;XP_046645

In [4]:
!zip -r blast_result_plots.zip blast_result_plots

  adding: blast_result_plots/ (stored 0%)
  adding: blast_result_plots/XP_035715441.1/ (stored 0%)
  adding: blast_result_plots/XP_035715441.1/round2_plot.svg (deflated 93%)
  adding: blast_result_plots/XP_035715441.1/round1_plot.svg (deflated 92%)
  adding: blast_result_plots/XP_035708168.1/ (stored 0%)
  adding: blast_result_plots/XP_035708168.1/round2_plot.svg (deflated 94%)
  adding: blast_result_plots/XP_035708168.1/round1_plot.svg (deflated 93%)
  adding: blast_result_plots/XP_015833841.1/ (stored 0%)
  adding: blast_result_plots/XP_015833841.1/round2_plot.svg (deflated 93%)
  adding: blast_result_plots/XP_015833841.1/round1_plot.svg (deflated 92%)
  adding: blast_result_plots/XP_034827763.1/ (stored 0%)
  adding: blast_result_plots/XP_034827763.1/round2_plot.svg (deflated 94%)
  adding: blast_result_plots/XP_034827763.1/round1_plot.svg (deflated 95%)
  adding: blast_result_plots/XP_023247543.1/ (stored 0%)
  adding: blast_result_plots/XP_023247543.1/round2_plot.svg (deflated 89%